# Checkpoint Three: Cleaning Data

Now you are ready to clean your data. Before starting coding, provide the link to your dataset below.

My dataset:

Import the necessary libraries and create your dataframe(s).

In [1]:
# Install dependencies as needed:

import pandas as pd
import numpy as np
import seaborn as sb
import matplotlib.pyplot as plt
from matplotlib import style

# pip install kagglehub[pandas-datasets]
import kagglehub
from kagglehub import KaggleDatasetAdapter

# Set the path to the file you'd like to load
file_path = "hotel_bookings.csv"

# Load the latest version
df = kagglehub.load_dataset(
  KaggleDatasetAdapter.PANDAS,
  "jessemostipak/hotel-booking-demand",
  file_path,
  # Provide any additional arguments like 
  # sql_query or pandas_kwargs. See the 
  # documenation for more information:
  # https://github.com/Kaggle/kagglehub/blob/main/README.md#kaggledatasetadapterpandas
)

print("First 5 records:", df.head())

/var/folders/ml/pkr27nwj14j_z_0bgf5tf4x40000gn/T/ipykernel_5567/985857877.py:17: DeprecationWarning: Use dataset_load() instead of load_dataset(). load_dataset() will be removed in a future version.
  df = kagglehub.load_dataset(


First 5 records:           hotel  is_canceled  lead_time  arrival_date_year arrival_date_month  \
0  Resort Hotel            0        342               2015               July   
1  Resort Hotel            0        737               2015               July   
2  Resort Hotel            0          7               2015               July   
3  Resort Hotel            0         13               2015               July   
4  Resort Hotel            0         14               2015               July   

   arrival_date_week_number  arrival_date_day_of_month  \
0                        27                          1   
1                        27                          1   
2                        27                          1   
3                        27                          1   
4                        27                          1   

   stays_in_weekend_nights  stays_in_week_nights  adults  ...  deposit_type  \
0                        0                     0       2  ...    No 

## Missing Data

Test your dataset for missing data and handle it as needed. Make notes in the form of code comments as to your thought process.

In [2]:
# I intially wanted to see exactly which rows had missing values and how many
df.isnull().sum()

# I concluded that the agent column was not necessary, so I dropped it
df.drop(columns="agent")

# I filled the missing values in the company and coutry columns w/unknown instead 
# of dropping them. They both provide meaningful categorical data
df['country'] = df['country'].fillna('Unknown')
df['company'] = df['company'].fillna('Unknown')

## Irregular Data

Detect outliers in your dataset and handle them as needed. Use code comments to make notes about your thought process.

In [6]:
# Businees question: “Which factors are most responsible for customer cancellations, 
# and how can we recommend improvements to reduce them?” By analyze hotel booking trends 
# by month, customer type, and cancellation rate...

# So, I should analyze lead_time, stays_in_weekend_nights, stay_in_week_nights, adults, children, 
# babies, previous_cancellations, booking_changes, and adr for outliers

# lead_time
df['lead_time'].describe()
# Lead time shows several values that have right-skewed distribution
# The maximum lead time is significantly higher than the median. This is an outlier.
# However, long lead times are valid in hotel bookings due to early planners or group reservations.
# So this column will be retained without any changes.

# stays_in_weekend_nights
df['stays_in_weekend_nights'].describe()
# Weekend night stays are right-skewed, with most bookings between 0 and 2 nights
# which is within normal range. While a small number of bookings show high values, 
# these represent valid data such as extended stays.

# stay_in_week_nights
df['stays_in_week_nights'].describe()
# This column shows a right-skewed distribution.
# Most bookings include 1–3 weeknights, with a small number of long-term stays.
# These higher values are valid and reflect extended or even business travel.

# adults
df['adults'].describe()
# The number of adults per booking is right-skewed. Most bookings are for one or 
# two adults, with a small number of group bookings. the median is a better 
# representation of a typical booking than the mean.

# children
df['children'].describe()
# The children column shows most bookings having no children. There's A small number 
# of family bookings create a right-skewed tail. The averages alone are misleading, and 
# the median or categorical grouping is more informative.

# babies
df['babies'].describe()
# The babies column demonstartes extremely low data dn renders useless in this analysis
# based on the statistical results.
# I am going to drop it.
df['babies'].dropna()

# previous_cancellations
df['previous_cancellations'].describe()
# Most guests never cancel. A small group cancels repeatedly. 

# booking_changes
df['booking_changes'].describe()
# Most guests never change their booking.
# A small subset changes their booking multiple times.

# adr
df['adr'].describe()
# ADR is a key revenue metric. There are negative ADR values were 
# identified is the statistical wanalysis. They are likely  refunds or data-entry anomalies 
# that will be addressed
df = df[df["adr"] >= 0]
df["adr"].describe()
# negative values removed

count    119389.000000
mean        101.832028
std          50.535032
min           0.000000
25%          69.290000
50%          94.590000
75%         126.000000
max        5400.000000
Name: adr, dtype: float64

## Unnecessary Data

Look for the different types of unnecessary data in your dataset and address it as needed. Make sure to use code comments to illustrate your thought process.

In [7]:
df.drop(columns='deposit_type')
df.drop(columns='company')
df.drop(columns='assigned_room_type')
df.drop(columns='agent')

# None of these aid in helping me answer my business question

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,...,booking_changes,deposit_type,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date
0,Resort Hotel,0,342,2015,July,27,1,0,0,2,...,3,No Deposit,Unknown,0,Transient,0.00,0,0,Check-Out,2015-07-01
1,Resort Hotel,0,737,2015,July,27,1,0,0,2,...,4,No Deposit,Unknown,0,Transient,0.00,0,0,Check-Out,2015-07-01
2,Resort Hotel,0,7,2015,July,27,1,0,1,1,...,0,No Deposit,Unknown,0,Transient,75.00,0,0,Check-Out,2015-07-02
3,Resort Hotel,0,13,2015,July,27,1,0,1,1,...,0,No Deposit,Unknown,0,Transient,75.00,0,0,Check-Out,2015-07-02
4,Resort Hotel,0,14,2015,July,27,1,0,2,2,...,0,No Deposit,Unknown,0,Transient,98.00,0,1,Check-Out,2015-07-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119385,City Hotel,0,23,2017,August,35,30,2,5,2,...,0,No Deposit,Unknown,0,Transient,96.14,0,0,Check-Out,2017-09-06
119386,City Hotel,0,102,2017,August,35,31,2,5,3,...,0,No Deposit,Unknown,0,Transient,225.43,0,2,Check-Out,2017-09-07
119387,City Hotel,0,34,2017,August,35,31,2,5,2,...,0,No Deposit,Unknown,0,Transient,157.71,0,4,Check-Out,2017-09-07
119388,City Hotel,0,109,2017,August,35,31,2,5,2,...,0,No Deposit,Unknown,0,Transient,104.40,0,0,Check-Out,2017-09-07


## Inconsistent Data

Check for inconsistent data and address any that arises. As always, use code comments to illustrate your thought process.

In [ ]:
# Inspecting any inconsistencies
df['customer_type'].unique()
df['market_segment'].unique()
df['distribution_channel'].unique()
df['meal'].unique()
df['deposit_type'].unique()
df['reservation_status'].unique()

# Must standardized text
categorical_cols = [
    'customer_type',
    'market_segment',
    'distribution_channel',
    'meal',
    'deposit_type',
    'reservation_status'
]

df[categorical_cols] = df[categorical_cols].apply(lambda x: x.str.strip().str.title())

# Update placeholders
df['meal'] = df['meal'].replace({'Undefined': 'Unknown'})

for col in categorical_cols:
    print(col, df[col].unique())

# I pretty much checked any categorical features for 
# inconsistencies using value count and unique values
# I standardized casing and removed placeholders 

customer_type ['Transient' 'Contract' 'Transient-Party' 'Group']
market_segment ['Direct' 'Corporate' 'Online Ta' 'Offline Ta/To' 'Complementary' 'Groups'
 'Undefined' 'Aviation']
distribution_channel ['Direct' 'Corporate' 'Ta/To' 'Undefined' 'Gds']
meal ['Bb' 'Fb' 'Hb' 'Sc' 'Unknown']
deposit_type ['No Deposit' 'Refundable' 'Non Refund']
reservation_status ['Check-Out' 'Canceled' 'No-Show']


## Summarize Your Results

Make note of your answers to the following questions.

1. Did you find all four types of dirty data in your dataset?
- Yes, the dataset contained missing values, inconsistent data, outliers, and unnecessary columns. Each of which were addressed through removal, stadardization, or exclusion based on relevance.

2. Did the process of cleaning your data give you new insights into your dataset?
- Yes, cleaning revealed that most customers do not cancel, while a small subset drive the majority of cancellations by customer type and booking behavior

3. Is there anything you would like to make note of when it comes to manipulating the data and making visualizations?
- Yes, cleaning must happen prior to aggregation and visualization. That way decisions can align with the business question to avoid misleading results.